### Generating Train-Val Split from Dataset

In [ ]:
import os
import shutil
import random
import multiprocessing
from copy import deepcopy

def recursive_copy_dicom(src_folder, dest_folder, file_counter):
    """
    Recursively finds and copies DICOM files from the source to the destination folder, renaming them sequentially.
    
    :param src_folder: The source folder containing DICOM files (including subdirectories).
    :param dest_folder: The destination folder where the files will be copied and renamed.
    :param file_counter: The sequential counter for renaming files.
    :return: List of renamed files for further splitting.
    """
    renamed_files = []

    for root, dirs, files in os.walk(src_folder):
        for dicom_file in files:
            if dicom_file.lower().endswith('.dcm'):
                # Get full path of the source file
                src_file_path = os.path.join(root, dicom_file)
                
                # Create the new file path in the destination folder
                dest_file_path = os.path.join(dest_folder, f"{file_counter}.dcm")
                
                # Copy and rename the file
                shutil.copy(src_file_path, dest_file_path)
                
                # Append the renamed file to the list
                renamed_files.append(f"{file_counter}.dcm")
                
                # Increment the file counter for the next file
                file_counter += 1

    return renamed_files

def split_and_transfer_files(file_list, dest_folder, split_factor):
    """
    Splits the list of renamed files into train and val sets and moves them into the appropriate folders.
    
    :param file_list: List of renamed DICOM files.
    :param dest_folder: Destination folder where train and val subfolders will be created.
    :param split_factor: The ratio of files to go into the train subfolder.
    """
    # Ensure the destination folder and subfolders exist
    train_folder = os.path.join(dest_folder, 'train')
    val_folder = os.path.join(dest_folder, 'val')
    
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    
    if not os.path.exists(val_folder):
        os.makedirs(val_folder)

    # Shuffle the files for randomness
    random.shuffle(file_list)

    # Calculate the number of files for the train and validation sets
    split_index = int(len(file_list) * split_factor)
    
    # Split the files into train and val sets
    train_files = file_list[:split_index]
    val_files = file_list[split_index:]

    # Move the files to the respective folders
    for file in train_files:
        src_file = os.path.join(dest_folder, file)
        dest_file = os.path.join(train_folder, file)
        shutil.move(src_file, dest_file)
        print(f"Moved {file} to train folder")
    
    for file in val_files:
        src_file = os.path.join(dest_folder, file)
        dest_file = os.path.join(val_folder, file)
        shutil.move(src_file, dest_file)
        print(f"Moved {file} to val folder")

def process_dicom_files(src_folder, dest_folder, split_factor):
    """
    Recursively finds, renames, copies DICOM files, and splits them into train and val sets.
    
    :param src_folder: The source folder containing DICOM files (including subdirectories).
    :param dest_folder: The destination folder where the renamed files and the train/val split will be created.
    :param split_factor: The ratio of files to go into the train subfolder.
    """
    # Ensure the destination folder exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Initialize file counter
    file_counter = 1

    # Recursively copy DICOM files and rename them
    renamed_files = recursive_copy_dicom(src_folder, dest_folder, file_counter)

    # Step 2: Split the renamed files into train and val sets
    split_and_transfer_files(renamed_files, dest_folder, split_factor)

# Example usage:
src_folder = r"F:\TCIA"  # Replace with your source folder path
dest_folder = r"F:\TCIA_Split"  # Destination folder for the renamed files and train/val split
split_factor = 0.9  # 90% of files will go to 'train', 10% will go to 'val'

# Perform the entire process
process_dicom_files(src_folder, dest_folder, split_factor)

### Basic U-Net

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from torch.utils.checkpoint import checkpoint
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MedicalImageDataset(Dataset):
    def __init__(self, dicom_dir):
        self.dicom_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    
    def __len__(self):
        return len(self.dicom_files)
    
    def __getitem__(self, idx):
        # Read DICOM file and normalize
        dcm = pydicom.dcmread(self.dicom_files[idx])
        image = dcm.pixel_array.astype(float)
        image = (image - image.min()) / (image.max() - image.min())
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)
        return image_tensor, image_tensor

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Encoder
        self.enc1 = UNetBlock(in_channels, 64)
        self.enc2 = UNetBlock(64, 128)
        self.enc3 = UNetBlock(128, 256)
        
        # Decoder
        self.dec3 = UNetBlock(256 + 128, 128)  # Adjust for concatenation with skip connection
        self.dec2 = UNetBlock(128 + 64, 64)    # Adjust for concatenation with skip connection
        self.dec1 = UNetBlock(64, out_channels)
        
        # Pooling and upsampling
        self.pool = nn.MaxPool2d(2, 2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    
    def forward(self, x):
        # Encoder path
        e1 = checkpoint(self.enc1, x)
        e2 = checkpoint(self.enc2, self.pool(e1))
        e3 = checkpoint(self.enc3, self.pool(e2))
        
        # Decoder path with skip connections
        d3 = self.upsample(e3)
        d3 = torch.cat([d3, e2], dim=1)  # Concatenate along channels
        d3 = checkpoint(self.dec3, d3)
        
        d2 = self.upsample(d3)
        d2 = torch.cat([d2, e1], dim=1)  # Concatenate along channels
        d2 = checkpoint(self.dec2, d2)
        
        d1 = self.dec1(d2)  # No checkpointing for final output layer
        
        return d1

def calculate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

def calculate_psnr(output, target, max_pixel=1.0):
    # Ensure the values are in the correct range
    mse = F.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def calculate_loss_and_psnr(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_psnr = 0
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            # Calculate MSE loss
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            # Calculate PSNR
            psnr = calculate_psnr(outputs, targets)
            total_psnr += psnr
    
    avg_loss = total_loss / num_batches
    avg_psnr = total_psnr / num_batches
    
    return avg_loss, avg_psnr

best_val_loss = float('inf')
best_model_path = 'best_model.pth'

def train_unet(dicom_dir, val_dicom_dir, epochs=50, batch_size=4, grad_accumulation_steps=2):
    # Dataset and DataLoader
    dataset = MedicalImageDataset(dicom_dir)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataset = MedicalImageDataset(val_dicom_dir)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Model, Loss, Optimizer
    model = UNet().to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop with tqdm
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        
        with tqdm(train_dataloader, unit="batch", desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
            for i, (images, targets) in enumerate(tepoch):
                images, targets = images.to(device), targets.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, targets)
                loss.backward()
                
                # Gradient accumulation
                if (i + 1) % grad_accumulation_steps == 0 or (i + 1) == len(tepoch):
                    optimizer.step()
                    optimizer.zero_grad()
                
                total_loss += loss.item()
                
                # Update the tqdm progress bar with the current loss
                tepoch.set_postfix(loss=total_loss / ((i + 1) * batch_size))
        
        avg_train_loss = total_loss / len(train_dataloader)
        avg_val_loss, avg_val_psnr = calculate_loss_and_psnr(model, val_dataloader, criterion)
        
        print(f"Epoch [{epoch+1}/{epochs}] - Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation PSNR: {avg_val_psnr:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"Model saved with improved validation loss: {avg_val_loss:.4f}")
    
    return model

# Example usage with train and validation directories
model = train_unet(r"D:\PN_Split\train", r"D:\PN_Split\val", epochs=50, batch_size=4, grad_accumulation_steps=8)

### U-Net Inference

In [ ]:
import torch
import torch.nn as nn
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import os

# Import the UNet and related classes from the previous script
# Replace with the actual import method

def load_dicom_image(dicom_path):
    """
    Load and normalize a DICOM image
    
    Args:
        dicom_path (str): Path to the DICOM file
    
    Returns:
        torch.Tensor: Normalized image tensor
    """
    # Read DICOM file
    dcm = pydicom.dcmread(dicom_path)
    image = dcm.pixel_array.astype(float)
    
    # Normalize image
    image = (image - image.min()) / (image.max() - image.min())
    
    # Convert to tensor
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    return image_tensor

def calculate_psnr(output, target, max_pixel=1.0):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR)
    
    Args:
        output (torch.Tensor): Reconstructed image
        target (torch.Tensor): Original image
        max_pixel (float): Maximum pixel value
    
    Returns:
        float: PSNR value
    """
    # Ensure the values are in the correct range
    mse = torch.nn.functional.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def visualize_reconstruction(original_image, reconstructed_image, psnr):
    """
    Visualize original and reconstructed images
    
    Args:
        original_image (torch.Tensor): Original image tensor
        reconstructed_image (torch.Tensor): Reconstructed image tensor
        psnr (float): Peak Signal-to-Noise Ratio
    """
    # Convert tensors to numpy for visualization
    original = original_image.squeeze().cpu().numpy()
    reconstructed = reconstructed_image.squeeze().cpu().numpy()
    
    # Create subplot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot original image
    im1 = ax1.imshow(original, cmap='gray')
    ax1.set_title('Original Image')
    plt.colorbar(im1, ax=ax1)
    
    # Plot reconstructed image
    im2 = ax2.imshow(reconstructed, cmap='gray')
    ax2.set_title(f'Reconstructed Image\nPSNR: {psnr:.2f} dB')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

def inference_single_image(model_path, test_dicom_path):
    """
    Perform inference on a single DICOM image
    
    Args:
        model_path (str): Path to the saved model weights
        test_dicom_path (str): Path to the test DICOM file
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize model
    model = UNet().to(device)
    
    # Load saved model weights
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Load and preprocess test image
    with torch.no_grad():
        test_image = load_dicom_image(test_dicom_path).to(device)
        
        # Perform reconstruction
        reconstructed_image = model(test_image)
        
        # Calculate PSNR
        psnr = calculate_psnr(reconstructed_image, test_image)

        print(f"PSNR: {psnr:.2f} dB")
        
        # Visualize results
        visualize_reconstruction(test_image, reconstructed_image, psnr)

# Example usage
if __name__ == "__main__":
    # Paths to model and test image
    MODEL_PATH = r"D:\VSCODE\PreSense\best_model.pth"  # Path to your saved model
    TEST_DICOM_PATH = r"D:\VSCODE\PreSense\test.dcm"  # Replace with actual path to test DICOM
    
    # Run inference
    inference_single_image(MODEL_PATH, TEST_DICOM_PATH)

### U-Net Inference for Complete Scan

In [ ]:
import torch
import torch.nn as nn
import pydicom
import numpy as np
import os
from tqdm import tqdm

# Import the UNet and related classes from the previous script

def load_dicom_image(dicom_path):
    """
    Load and normalize a DICOM image
    
    Args:
        dicom_path (str): Path to the DICOM file
    
    Returns:
        torch.Tensor: Normalized image tensor
    """
    # Read DICOM file
    dcm = pydicom.dcmread(dicom_path)
    image = dcm.pixel_array.astype(float)
    
    # Normalize image
    image = (image - image.min()) / (image.max() - image.min())
    
    # Convert to tensor
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    return image_tensor, dcm

def save_reconstructed_dicom(image_tensor, original_dcm, output_path):
    """
    Save reconstructed image as a DICOM file
    
    Args:
        image_tensor (torch.Tensor): Reconstructed image tensor
        original_dcm (pydicom.Dataset): Original DICOM dataset
        output_path (str): Path to save the reconstructed image
    """
    # Convert tensor to numpy and scale back to original pixel range
    reconstructed_image = image_tensor.squeeze().cpu().numpy()
    
    # Scale to original pixel array range
    min_val = original_dcm.pixel_array.min()
    max_val = original_dcm.pixel_array.max()
    reconstructed_image = reconstructed_image * (max_val - min_val) + min_val
    
    # Create a copy of the original DICOM dataset
    ds = pydicom.Dataset()
    ds.update(original_dcm)
    
    # Set the new pixel data
    ds.PixelData = reconstructed_image.astype(original_dcm.pixel_array.dtype).tobytes()
    
    # Set transfer syntax to explicit VR little endian (common default)
    ds.file_meta = pydicom.Dataset()
    ds.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian
    
    # Write the DICOM file
    pydicom.dcmwrite(output_path, ds)

def calculate_psnr(output, target, max_pixel=1.0):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR)
    
    Args:
        output (torch.Tensor): Reconstructed image
        target (torch.Tensor): Original image
        max_pixel (float): Maximum pixel value
    
    Returns:
        float: PSNR value
    """
    # Ensure the values are in the correct range
    mse = torch.nn.functional.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def batch_inference(model_path, input_dir, output_dir):
    """
    Perform batch inference on all DICOM files in a directory
    
    Args:
        model_path (str): Path to the saved model weights
        input_dir (str): Directory containing input DICOM files
        output_dir (str): Directory to save reconstructed DICOM files
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize model
    model = UNet().to(device)
    
    # Load saved model weights
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Get list of DICOM files
    dcm_files = [f for f in os.listdir(input_dir) if f.endswith('.dcm')]
    
    # Prepare for inference
    print(f"Starting batch inference on {len(dcm_files)} DICOM files...")
    
    # Store PSNR values
    psnr_values = {}
    
    # Perform inference
    with torch.no_grad():
        for dcm_file in tqdm(dcm_files, desc="Reconstructing Images"):
            # Full paths
            input_path = os.path.join(input_dir, dcm_file)
            output_path = os.path.join(output_dir, dcm_file)
            
            # Load image
            test_image, original_dcm = load_dicom_image(input_path)
            test_image = test_image.to(device)
            
            # Perform reconstruction
            reconstructed_image = model(test_image)
            
            # Calculate PSNR
            psnr = calculate_psnr(reconstructed_image, test_image)
            psnr_values[dcm_file] = psnr
            
            # Save reconstructed image
            save_reconstructed_dicom(reconstructed_image, original_dcm, output_path)
    
    # Print PSNR values
    print("\nPSNR Values:")
    for filename, psnr in psnr_values.items():
        print(f"{filename}: {psnr:.2f} dB")
    
    # Calculate and print overall statistics
    psnr_list = list(psnr_values.values())
    print(f"\nPSNR Statistics:")
    print(f"Average PSNR: {np.mean(psnr_list):.2f} dB")
    print(f"Minimum PSNR: {np.min(psnr_list):.2f} dB")
    print(f"Maximum PSNR: {np.max(psnr_list):.2f} dB")

# Example usage
if __name__ == "__main__":
    # Paths to model, input, and output directories
    MODEL_PATH = r"D:\VSCODE\PreSense\best_model.pth"  # Path to your saved model
    INPUT_DICOM_DIR = r"D:\Pancreatic Neuroendocrine\manifest-1662644254281\CTpred-Sunitinib-panNET\PAN_01\04-11-2001-NA-NA-29221\3.000000-CEFC07AIDR 3D STD-16260"  # Directory with input DICOM files
    OUTPUT_DICOM_DIR = r"D:\VSCODE\PreSense\reconstructed_dicom"  # Directory to save reconstructed DICOM files
    
    # Run batch inference
    batch_inference(MODEL_PATH, INPUT_DICOM_DIR, OUTPUT_DICOM_DIR)

In [ ]:
import torch
import torch.nn as nn
import pydicom
import numpy as np
import os
from tqdm import tqdm
from PIL import Image

# Import the UNet and related classes from the previous script

def load_dicom_image(dicom_path):
    """
    Load and normalize a DICOM image
    
    Args:
        dicom_path (str): Path to the DICOM file
    
    Returns:
        torch.Tensor: Normalized image tensor
    """
    # Read DICOM file
    dcm = pydicom.dcmread(dicom_path)
    image = dcm.pixel_array.astype(float)
    
    # Normalize image
    image = (image - image.min()) / (image.max() - image.min())
    
    # Convert to tensor
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    return image_tensor, dcm

def save_reconstructed_image(image_tensor, output_path):
    """
    Save reconstructed image as a JPEG file
    
    Args:
        image_tensor (torch.Tensor): Reconstructed image tensor
        output_path (str): Path to save the reconstructed JPEG image
    """
    # Convert tensor to numpy array
    reconstructed_image = image_tensor.squeeze().cpu().numpy()
    
    # Scale back to the original pixel range (assuming input was normalized to [0, 1])
    reconstructed_image = np.uint8(reconstructed_image * 255)
    
    # Convert to PIL Image
    pil_image = Image.fromarray(reconstructed_image)
    
    # Save as JPEG
    pil_image.save(output_path, 'JPEG')

def calculate_psnr(output, target, max_pixel=1.0):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR)
    
    Args:
        output (torch.Tensor): Reconstructed image
        target (torch.Tensor): Original image
        max_pixel (float): Maximum pixel value
    
    Returns:
        float: PSNR value
    """
    # Ensure the values are in the correct range
    mse = torch.nn.functional.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def batch_inference(model_path, input_dir, output_dir):
    """
    Perform batch inference on all DICOM files in a directory
    
    Args:
        model_path (str): Path to the saved model weights
        input_dir (str): Directory containing input DICOM files
        output_dir (str): Directory to save reconstructed JPEG images
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize model
    model = UNet().to(device)
    
    # Load saved model weights
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # Get list of DICOM files
    dcm_files = [f for f in os.listdir(input_dir) if f.endswith('.dcm')]
    
    # Prepare for inference
    print(f"Starting batch inference on {len(dcm_files)} DICOM files...")
    
    # Store PSNR values
    psnr_values = {}
    
    # Perform inference
    with torch.no_grad():
        for dcm_file in tqdm(dcm_files, desc="Reconstructing Images"):
            # Full paths
            input_path = os.path.join(input_dir, dcm_file)
            output_path = os.path.join(output_dir, f"{os.path.splitext(dcm_file)[0]}.jpg")  # Save as .jpg
            
            # Load image
            test_image, original_dcm = load_dicom_image(input_path)
            test_image = test_image.to(device)
            
            # Perform reconstruction
            reconstructed_image = model(test_image)
            
            # Calculate PSNR
            psnr = calculate_psnr(reconstructed_image, test_image)
            psnr_values[dcm_file] = psnr
            
            # Save reconstructed image as JPEG
            save_reconstructed_image(reconstructed_image, output_path)
    
    # Print PSNR values
    print("\nPSNR Values:")
    for filename, psnr in psnr_values.items():
        print(f"{filename}: {psnr:.2f} dB")
    
    # Calculate and print overall statistics
    psnr_list = list(psnr_values.values())
    print(f"\nPSNR Statistics:")
    print(f"Average PSNR: {np.mean(psnr_list):.2f} dB")
    print(f"Minimum PSNR: {np.min(psnr_list):.2f} dB")
    print(f"Maximum PSNR: {np.max(psnr_list):.2f} dB")

# Example usage
if __name__ == "__main__":
    # Paths to model, input, and output directories
    MODEL_PATH = r"D:\VSCODE\PreSense\best_model.pth"  # Path to your saved model
    INPUT_DICOM_DIR = r"D:\Pancreatic Neuroendocrine\manifest-1662644254281\CTpred-Sunitinib-panNET\PAN_01\04-11-2001-NA-NA-29221\3.000000-CEFC07AIDR 3D STD-16260"  # Directory with input DICOM files
    OUTPUT_JPEG_DIR = r"D:\VSCODE\PreSense\reconstructed_images"  # Directory to save reconstructed JPEG images
    
    # Run batch inference
    batch_inference(MODEL_PATH, INPUT_DICOM_DIR, OUTPUT_JPEG_DIR)

### Small Reconstructor and Denoiser U-Net (smallRD)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from torch.utils.checkpoint import checkpoint
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MedicalImageDataset(Dataset):
    def __init__(self, dicom_dir):
        self.dicom_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    
    def __len__(self):
        return len(self.dicom_files)
    
    def __getitem__(self, idx):
        # Read DICOM file and normalize
        dcm = pydicom.dcmread(self.dicom_files[idx])
        image = dcm.pixel_array.astype(float)
        image = (image - image.min()) / (image.max() - image.min())
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)
        return image_tensor, image_tensor

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Encoder
        self.enc1 = UNetBlock(in_channels, 64)
        self.enc2 = UNetBlock(64, 128)
        self.enc3 = UNetBlock(128, 256)
        
        # Decoder
        self.dec3 = UNetBlock(256 + 128, 128)  # Adjust for concatenation with skip connection
        self.dec2 = UNetBlock(128 + 64, 64)    # Adjust for concatenation with skip connection
        self.dec1 = UNetBlock(64, out_channels)
        
        # Pooling and upsampling
        self.pool = nn.MaxPool2d(2, 2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    
    def forward(self, x):
        # Encoder path
        e1 = checkpoint(self.enc1, x)
        e2 = checkpoint(self.enc2, self.pool(e1))
        e3 = checkpoint(self.enc3, self.pool(e2))
        
        # Decoder path with skip connections
        d3 = self.upsample(e3)
        d3 = torch.cat([d3, e2], dim=1)  # Concatenate along channels
        d3 = checkpoint(self.dec3, d3)
        
        d2 = self.upsample(d3)
        d2 = torch.cat([d2, e1], dim=1)  # Concatenate along channels
        d2 = checkpoint(self.dec2, d2)
        
        d1 = self.dec1(d2)  # No checkpointing for final output layer
        
        return d1

def calculate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

def calculate_psnr(output, target, max_pixel=1.0):
    # Ensure the values are in the correct range
    mse = F.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def calculate_loss_and_psnr(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_psnr = 0
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            # Calculate MSE loss
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            # Calculate PSNR
            psnr = calculate_psnr(outputs, targets)
            total_psnr += psnr
    
    avg_loss = total_loss / num_batches
    avg_psnr = total_psnr / num_batches
    
    return avg_loss, avg_psnr

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Encoder
        self.enc1 = UNetBlock(in_channels, 64)
        self.enc2 = UNetBlock(64, 128)
        self.enc3 = UNetBlock(128, 256)
        
        # Decoder
        self.dec3 = UNetBlock(256 + 128, 128)  # Adjust for concatenation with skip connection
        self.dec2 = UNetBlock(128 + 64, 64)    # Adjust for concatenation with skip connection
        self.dec1 = UNetBlock(64, out_channels)
        
        # Pooling and upsampling
        self.pool = nn.MaxPool2d(2, 2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
    
    def forward(self, x):
        # Encoder path
        e1 = checkpoint(self.enc1, x)
        e2 = checkpoint(self.enc2, self.pool(e1))
        e3 = checkpoint(self.enc3, self.pool(e2))
        
        # Decoder path with skip connections
        d3 = self.upsample(e3)
        d3 = torch.cat([d3, e2], dim=1)  # Concatenate along channels
        d3 = checkpoint(self.dec3, d3)
        
        d2 = self.upsample(d3)
        d2 = torch.cat([d2, e1], dim=1)  # Concatenate along channels
        d2 = checkpoint(self.dec2, d2)
        
        d1 = self.dec1(d2)  # No checkpointing for final output layer
        
        return d1

class Reconstructor(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Same UNet architecture for reconstruction
        self.unet = UNet(in_channels=in_channels, out_channels=out_channels)
    
    def forward(self, x):
        return self.unet(x)


class Denoiser(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Same UNet architecture for denoising
        self.unet = UNet(in_channels=in_channels, out_channels=out_channels)
    
    def forward(self, x):
        return self.unet(x)
    
def train_reconstructor_and_denoiser(dicom_dir, val_dicom_dir, epochs=50, batch_size=4, grad_accumulation_steps=2):
    # Dataset and DataLoader
    dataset = MedicalImageDataset(dicom_dir)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataset = MedicalImageDataset(val_dicom_dir)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize both models
    reconstructor = Reconstructor().to(device)
    denoiser = Denoiser().to(device)
    
    # Loss functions for both models
    reconstructor_criterion = nn.MSELoss()
    denoiser_criterion = nn.MSELoss()
    
    # Optimizers for both models
    reconstructor_optimizer = torch.optim.Adam(reconstructor.parameters(), lr=0.0001)
    denoiser_optimizer = torch.optim.Adam(denoiser.parameters(), lr=0.0001)
    
    # Best validation loss initialization
    best_reconstructor_val_loss = float('inf')
    best_denoiser_val_loss = float('inf')
    best_reconstructor_model_path = 'best_reconstructor_model.pth'
    best_denoiser_model_path = 'best_denoiser_model.pth'

    # Training loop with tqdm
    for epoch in range(epochs):
        reconstructor.train()
        denoiser.train()
        
        reconstructor_total_loss = 0
        denoiser_total_loss = 0
        
        reconstructor_optimizer.zero_grad()
        denoiser_optimizer.zero_grad()

        with tqdm(train_dataloader, unit="batch", desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
            for i, (images, targets) in enumerate(tepoch):
                images, targets = images.to(device), targets.to(device)
                
                # Training Reconstructor
                reconstructor_outputs = reconstructor(images)
                reconstructor_loss = reconstructor_criterion(reconstructor_outputs, targets)
                reconstructor_loss.backward(retain_graph=True)

                # Gradient accumulation for reconstructor
                if (i + 1) % grad_accumulation_steps == 0 or (i + 1) == len(tepoch):
                    reconstructor_optimizer.step()
                    reconstructor_optimizer.zero_grad()

                reconstructor_total_loss += reconstructor_loss.item()

                # Training Denoiser (using output from Reconstructor as noisy input)
                noisy_images = reconstructor_outputs.detach()  # Detach from the computation graph to avoid in-place error
                denoiser_outputs = denoiser(noisy_images)
                denoiser_loss = denoiser_criterion(denoiser_outputs, targets)
                denoiser_loss.backward()

                # Gradient accumulation for denoiser
                if (i + 1) % grad_accumulation_steps == 0 or (i + 1) == len(tepoch):
                    denoiser_optimizer.step()
                    denoiser_optimizer.zero_grad()

                denoiser_total_loss += denoiser_loss.item()

                # Update the tqdm progress bar with current loss
                tepoch.set_postfix(
                    reconstructor_loss=reconstructor_total_loss / ((i + 1) * batch_size),
                    denoiser_loss=denoiser_total_loss / ((i + 1) * batch_size)
                )
        
        # Calculate validation loss for both models
        avg_reconstructor_train_loss = reconstructor_total_loss / len(train_dataloader)
        avg_denoiser_train_loss = denoiser_total_loss / len(train_dataloader)
        
        avg_reconstructor_val_loss, avg_reconstructor_val_psnr = calculate_loss_and_psnr(reconstructor, val_dataloader, reconstructor_criterion)
        avg_denoiser_val_loss, avg_denoiser_val_psnr = calculate_loss_and_psnr(denoiser, val_dataloader, denoiser_criterion)
        
        print(f"Epoch [{epoch+1}/{epochs}] - "
              f"Reconstructor Train Loss: {avg_reconstructor_train_loss:.4f}, "
              f"Denoiser Train Loss: {avg_denoiser_train_loss:.4f}, "
              f"Reconstructor Val Loss: {avg_reconstructor_val_loss:.4f}, "
              f"Denoiser Val Loss: {avg_denoiser_val_loss:.4f}, "
              f"Reconstructor Validation PSNR: {avg_reconstructor_val_psnr:.4f}, "
              f"Denoiser Validation PSNR: {avg_denoiser_val_psnr:.4f}")
        
        # Save models if validation loss is improved
        if avg_reconstructor_val_loss < best_reconstructor_val_loss:
            best_reconstructor_val_loss = avg_reconstructor_val_loss
            torch.save(reconstructor.state_dict(), best_reconstructor_model_path)
            print(f"Reconstructor model saved with improved validation loss: {avg_reconstructor_val_loss:.4f}")
        
        if avg_denoiser_val_loss < best_denoiser_val_loss:
            best_denoiser_val_loss = avg_denoiser_val_loss
            torch.save(denoiser.state_dict(), best_denoiser_model_path)
            print(f"Denoiser model saved with improved validation loss: {avg_denoiser_val_loss:.4f}")
    
    return reconstructor, denoiser

# Example usage with train and validation directories
reconstructor_model, denoiser_model = train_reconstructor_and_denoiser(
    r"D:/PN_Split/train", r"D:/PN_Split/val", epochs=50, batch_size=20, grad_accumulation_steps=2
)

### smallRD Single Image Inference

In [ ]:
import torch
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import os

# Import the models from the previous script
# Assuming they are defined or imported correctly

def load_dicom_image(dicom_path):
    """
    Load and normalize a DICOM image
    
    Args:
        dicom_path (str): Path to the DICOM file
    
    Returns:
        torch.Tensor: Normalized image tensor
    """
    # Read DICOM file
    dcm = pydicom.dcmread(dicom_path)
    image = dcm.pixel_array.astype(float)
    
    # Normalize image
    image = (image - image.min()) / (image.max() - image.min())
    
    # Convert to tensor
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions
    return image_tensor

def calculate_psnr(output, target, max_pixel=1.0):
    """
    Calculate Peak Signal-to-Noise Ratio (PSNR)
    
    Args:
        output (torch.Tensor): Reconstructed image
        target (torch.Tensor): Original image
        max_pixel (float): Maximum pixel value
    
    Returns:
        float: PSNR value
    """
    # Ensure the values are in the correct range
    mse = torch.nn.functional.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def visualize_reconstruction(original_image, reconstructed_image, psnr):
    """
    Visualize original and reconstructed images
    
    Args:
        original_image (torch.Tensor): Original image tensor
        reconstructed_image (torch.Tensor): Reconstructed image tensor
        psnr (float): Peak Signal-to-Noise Ratio
    """
    # Convert tensors to numpy for visualization
    original = original_image.squeeze().cpu().numpy()
    reconstructed = reconstructed_image.squeeze().cpu().numpy()
    
    # Create subplot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    
    # Plot original image
    im1 = ax1.imshow(original, cmap='gray')
    ax1.set_title('Original Image')
    plt.colorbar(im1, ax=ax1)
    
    # Plot reconstructed image
    im2 = ax2.imshow(reconstructed, cmap='gray')
    ax2.set_title(f'Reconstructed Image\nPSNR: {psnr:.2f} dB')
    plt.colorbar(im2, ax=ax2)
    
    plt.tight_layout()
    plt.show()

def inference_single_image(reconstructor_model_path, denoiser_model_path, test_dicom_path):
    """
    Perform inference on a single DICOM image using both Reconstructor and Denoiser models.
    
    Args:
        reconstructor_model_path (str): Path to the saved Reconstructor model weights
        denoiser_model_path (str): Path to the saved Denoiser model weights
        test_dicom_path (str): Path to the test DICOM file
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Initialize models
    reconstructor = Reconstructor().to(device)
    denoiser = Denoiser().to(device)
    
    # Load saved model weights
    reconstructor.load_state_dict(torch.load(reconstructor_model_path))
    denoiser.load_state_dict(torch.load(denoiser_model_path))
    
    reconstructor.eval()
    denoiser.eval()
    
    # Load and preprocess test image
    with torch.no_grad():
        test_image = load_dicom_image(test_dicom_path).to(device)
        
        # Perform reconstruction
        reconstructed_image = reconstructor(test_image)
        
        # Perform denoising on the reconstructed image
        denoised_image = denoiser(reconstructed_image)
        
        # Calculate PSNR for both original and denoised outputs
        psnr_reconstructed = calculate_psnr(reconstructed_image, test_image)
        psnr_denoised = calculate_psnr(denoised_image, test_image)

        print(f"PSNR (Reconstructed): {psnr_reconstructed:.2f} dB")
        print(f"PSNR (Denoised): {psnr_denoised:.2f} dB")
        
        # Visualize results
        visualize_reconstruction(test_image, reconstructed_image, psnr_reconstructed)
        visualize_reconstruction(test_image, denoised_image, psnr_denoised)

# Example usage
if __name__ == "__main__":
    # Paths to models and test image
    RECONSTRUCTOR_MODEL_PATH = r"D:/VSCODE/PreSense/small_reconstructor.pth"  # Path to your saved Reconstructor model
    DENOISER_MODEL_PATH = r"D:/VSCODE/PreSense/small_denoiser.pth"  # Path to your saved Denoiser model
    TEST_DICOM_PATH = r"D:/VSCODE/PreSense/test2.dcm"  # Replace with actual path to test DICOM    
    # Run inference
    inference_single_image(RECONSTRUCTOR_MODEL_PATH, DENOISER_MODEL_PATH, TEST_DICOM_PATH)

### Larger Reconstructor and Denoiser U-Net (largeRD)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os
from torch.utils.checkpoint import checkpoint
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class MedicalImageDataset(Dataset):
    def __init__(self, dicom_dir):
        self.dicom_files = [os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.dcm')]
    
    def __len__(self):
        return len(self.dicom_files)
    
    def __getitem__(self, idx):
        # Read DICOM file and normalize
        dcm = pydicom.dcmread(self.dicom_files[idx])
        image = dcm.pixel_array.astype(float)
        image = (image - image.min()) / (image.max() - image.min())
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image).float().unsqueeze(0)
        return image_tensor, image_tensor

class UNetBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        return x

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Encoder
        self.enc1 = UNetBlock(in_channels, 96)
        self.enc2 = UNetBlock(96, 192)
        self.enc3 = UNetBlock(192, 384)
        self.enc4 = UNetBlock(384, 784)
        
        # Decoder with learned upsampling (transposed convolutions)
        self.upconv4 = nn.ConvTranspose2d(784, 384, kernel_size=2, stride=2)  # Learnable upsampling
        self.dec4 = UNetBlock(384 + 384, 384)  # Adjust input channels after concatenation

        self.upconv3 = nn.ConvTranspose2d(384, 192, kernel_size=2, stride=2)  # Learnable upsampling
        self.dec3 = UNetBlock(192 + 192, 192)  # Adjust input channels after concatenation

        self.upconv2 = nn.ConvTranspose2d(192, 96, kernel_size=2, stride=2)  # Learnable upsampling
        self.dec2 = UNetBlock(96 + 96, 96)  # Adjust input channels after concatenation

        self.dec1 = UNetBlock(96, out_channels)  # Final output

        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        # Encoder path
        e1 = checkpoint(self.enc1, x)
        e2 = checkpoint(self.enc2, self.pool(e1))
        e3 = checkpoint(self.enc3, self.pool(e2))
        e4 = checkpoint(self.enc4, self.pool(e3))
        
        # Decoder path with learned upsampling and skip connections
        d4 = self.upconv4(e4)  # Learnable upsampling
        d4 = torch.cat([d4, e3], dim=1)  # Concatenate with encoder features
        d4 = checkpoint(self.dec4, d4)

        d3 = self.upconv3(d4)  # Learnable upsampling
        d3 = torch.cat([d3, e2], dim=1)  # Concatenate with encoder features
        d3 = checkpoint(self.dec3, d3)

        d2 = self.upconv2(d3)  # Learnable upsampling
        d2 = torch.cat([d2, e1], dim=1)  # Concatenate with encoder features
        d2 = checkpoint(self.dec2, d2)
        
        d1 = self.dec1(d2)  # No checkpointing for final output layer
        
        return d1

def calculate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    return total_loss / len(dataloader)

def calculate_psnr(output, target, max_pixel=1.0):
    # Ensure the values are in the correct range
    mse = F.mse_loss(output, target)
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr.item()

def calculate_loss_and_psnr(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    total_psnr = 0
    num_batches = len(dataloader)
    
    with torch.no_grad():
        for images, targets in dataloader:
            images, targets = images.to(device), targets.to(device)
            outputs = model(images)
            
            # Calculate MSE loss
            loss = criterion(outputs, targets)
            total_loss += loss.item()
            
            # Calculate PSNR
            psnr = calculate_psnr(outputs, targets)
            total_psnr += psnr
    
    avg_loss = total_loss / num_batches
    avg_psnr = total_psnr / num_batches
    
    return avg_loss, avg_psnr

class Reconstructor(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Same UNet architecture for reconstruction
        self.unet = UNet(in_channels=in_channels, out_channels=out_channels)
    
    def forward(self, x):
        return self.unet(x)


class Denoiser(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        # Same UNet architecture for denoising
        self.unet = UNet(in_channels=in_channels, out_channels=out_channels)
    
    def forward(self, x):
        return self.unet(x)
    
def train_reconstructor_and_denoiser(dicom_dir, val_dicom_dir, epochs=50, batch_size=4, grad_accumulation_steps=2):
    # Dataset and DataLoader
    dataset = MedicalImageDataset(dicom_dir)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    val_dataset = MedicalImageDataset(val_dicom_dir)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize both models
    reconstructor = Reconstructor().to(device)
    denoiser = Denoiser().to(device)
    
    # Loss functions for both models
    reconstructor_criterion = nn.MSELoss()
    denoiser_criterion = nn.MSELoss()
    
    # Optimizers for both models
    reconstructor_optimizer = torch.optim.Adam(reconstructor.parameters(), lr=0.0001)
    denoiser_optimizer = torch.optim.Adam(denoiser.parameters(), lr=0.0001)
    
    # Best validation loss initialization
    best_reconstructor_val_loss = float('inf')
    best_denoiser_val_loss = float('inf')
    best_reconstructor_model_path = 'best_reconstructor_model.pth'
    best_denoiser_model_path = 'best_denoiser_model.pth'

    # Training loop with tqdm
    for epoch in range(epochs):
        reconstructor.train()
        denoiser.train()
        
        reconstructor_total_loss = 0
        denoiser_total_loss = 0
        
        reconstructor_optimizer.zero_grad()
        denoiser_optimizer.zero_grad()

        with tqdm(train_dataloader, unit="batch", desc=f"Epoch {epoch+1}/{epochs}") as tepoch:
            for i, (images, targets) in enumerate(tepoch):
                images, targets = images.to(device), targets.to(device)
                
                # Training Reconstructor
                reconstructor_outputs = reconstructor(images)
                reconstructor_loss = reconstructor_criterion(reconstructor_outputs, targets)
                reconstructor_loss.backward(retain_graph=True)

                # Gradient accumulation for reconstructor
                if (i + 1) % grad_accumulation_steps == 0 or (i + 1) == len(tepoch):
                    reconstructor_optimizer.step()
                    reconstructor_optimizer.zero_grad()

                reconstructor_total_loss += reconstructor_loss.item()

                # Training Denoiser (using output from Reconstructor as noisy input)
                noisy_images = reconstructor_outputs.detach()  # Detach from the computation graph to avoid in-place error
                denoiser_outputs = denoiser(noisy_images)
                denoiser_loss = denoiser_criterion(denoiser_outputs, targets)
                denoiser_loss.backward()

                # Gradient accumulation for denoiser
                if (i + 1) % grad_accumulation_steps == 0 or (i + 1) == len(tepoch):
                    denoiser_optimizer.step()
                    denoiser_optimizer.zero_grad()

                denoiser_total_loss += denoiser_loss.item()

                # Update the tqdm progress bar with current loss
                tepoch.set_postfix(
                    reconstructor_loss=reconstructor_total_loss / ((i + 1) * batch_size),
                    denoiser_loss=denoiser_total_loss / ((i + 1) * batch_size)
                )
        
        # Calculate validation loss for both models
        avg_reconstructor_train_loss = reconstructor_total_loss / len(train_dataloader)
        avg_denoiser_train_loss = denoiser_total_loss / len(train_dataloader)
        
        avg_reconstructor_val_loss, _ = calculate_loss_and_psnr(reconstructor, val_dataloader, reconstructor_criterion)
        avg_denoiser_val_loss, _ = calculate_loss_and_psnr(denoiser, val_dataloader, denoiser_criterion)
        
        print(f"Epoch [{epoch+1}/{epochs}] - "
              f"Reconstructor Train Loss: {avg_reconstructor_train_loss:.4f}, "
              f"Denoiser Train Loss: {avg_denoiser_train_loss:.4f}, "
              f"Reconstructor Val Loss: {avg_reconstructor_val_loss:.4f}, "
              f"Denoiser Val Loss: {avg_denoiser_val_loss:.4f}")
        
        # Save models if validation loss is improved
        if avg_reconstructor_val_loss < best_reconstructor_val_loss:
            best_reconstructor_val_loss = avg_reconstructor_val_loss
            torch.save(reconstructor.state_dict(), best_reconstructor_model_path)
            print(f"Reconstructor model saved with improved validation loss: {avg_reconstructor_val_loss:.4f}")
        
        if avg_denoiser_val_loss < best_denoiser_val_loss:
            best_denoiser_val_loss = avg_denoiser_val_loss
            torch.save(denoiser.state_dict(), best_denoiser_model_path)
            print(f"Denoiser model saved with improved validation loss: {avg_denoiser_val_loss:.4f}")
    
    return reconstructor, denoiser

# Example usage with train and validation directories
reconstructor_model, denoiser_model = train_reconstructor_and_denoiser(
    r"D:\PN_Split\train", r"D:\PN_Split\val", epochs=50, batch_size=1, grad_accumulation_steps=64
)